# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [229]:
# your answer here
import pandas as pd
import numpy as np

# Challenge 1

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [230]:
# your answer here
employee = pd.read_csv('data\Employee.csv')
employee.head(3)

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [231]:
# Method 1
# your answer here
employee.groupby('Department').agg({'Salary':'mean'})

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [232]:
# Method 2
# your answer here
employee.pivot_table(index='Department', values='Salary')

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [233]:
# your answer here
employee.pivot_table(index=['Department','Title'], values='Salary')

Salary
Department Title            
HR         VP           70.0
           analyst      32.5
IT         VP           70.0
           analyst      32.5
           associate    60.0
Sales      associate    55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [234]:
# your answer here
employee.pivot_table(index=['Department','Title'], values=['Salary','Years'])

Salary  Years
Department Title                   
HR         VP           70.0    8.0
           analyst      32.5    3.0
IT         VP           70.0    8.0
           analyst      32.5    1.5
           associate    60.0    7.0
Sales      associate    55.0    2.5

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [235]:
# your answer here
employee.pivot_table(index=['Department','Title'], values=['Salary', 'Name'], aggfunc={'Salary':'mean','Name':'count'})

Name  Salary
Department Title                  
HR         VP            1    70.0
           analyst       2    32.5
IT         VP            1    70.0
           analyst       2    32.5
           associate     1    60.0
Sales      associate     2    55.0

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [236]:
# your answer here
salary_info = employee.pivot_table(index=['Department','Title'], values=['Salary'], aggfunc={'Salary':['median','min','max','std']})
salary_info

Salary                     
                        max median min       std
Department Title                                
HR         VP            70   70.0  70       NaN
           analyst       35   32.5  30  3.535534
IT         VP            70   70.0  70       NaN
           analyst       35   32.5  30  3.535534
           associate     60   60.0  60       NaN
Sales      associate     55   55.0  55  0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [237]:
# your answer here
salary_info['Salary','std'] = salary_info['Salary','std'].fillna(0)
salary_info

Salary                     
                        max median min       std
Department Title                                
HR         VP            70   70.0  70  0.000000
           analyst       35   32.5  30  3.535534
IT         VP            70   70.0  70  0.000000
           analyst       35   32.5  30  3.535534
           associate     60   60.0  60  0.000000
Sales      associate     55   55.0  55  0.000000

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [238]:
# your answer here
employee.pivot_table(index=['Department'], values=['Name','Salary'], aggfunc={'Name': 'count','Salary':'sum'})

,Name,Salary
Department,,
HR,3,135
IT,4,195
Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [239]:
# your answer here
employee.pivot_table(index=['Department','Title'], values=['Years','Salary'], aggfunc={'Years':'sum'})

Years
Department Title           
HR         VP             8
           analyst        6
IT         VP             8
           analyst        3
           associate      7
Sales      associate      5

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [240]:
# your answer here
max_salaries = employee.pivot_table(index=['Department'], values=['Salary'], aggfunc={'Salary':'max'})
maxsal = max_salaries['Salary'].values

def womax(row):
    if row['Salary'] not in maxsal:
        return True
    else:
        return False

employee['NoMax'] = employee.apply(womax, axis=1)
employee = employee[employee['NoMax']]

meanwomax = employee.pivot_table(index=['Department'], values=['Salary'], aggfunc={'Salary':'mean'})
meanwomax

,Salary
Department,
HR,32.500000
IT,41.666667


# Challenge 2

#### Open the ` Fitbit.csv` file and name your dataset `fitbit`.

In [241]:
# your answer here


#### Explore you dataset in terms of data types and descriptive statistics.
Hint: Use Pandas functions from previous lectures.

In [242]:
# your answer here


In [243]:
# your answer here


#### You suspect that there must be a linear relationship between the Minutes Very Active and the Steps. Compute the correlation between these variables.

In [244]:
# your answer here


#### Use matplotlib to visually plot Minutes Very active vs Steps. 

##### Hint : import matplotlib.pyplot as plt (See previous lessons)#####
- Use plt.scatter(x=df['col_name_1'], y=df['col_name_2'])

In [245]:
# your answer here


#### What can you say about Minute Very Active and Steps? Write a comment below

In [246]:
# your answer here


#### We also suspect that there must be a linear relationship between the Minutes Sedentary and the Steps. Compute the correlation between these variables.

In [247]:
# your answer here


#### Use matplotlib to visually plot Minutes Sedentary vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [248]:
# your answer here


#### We also suspect that there must be a linear relationship between the MinutesOfSleep and the Steps. Compute the correlation between these variables.

In [249]:
# your answer here


#### Use matplotlib to visually plot MinutesOfSleep vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [250]:
# your answer here


#### Select a subset of your dataset with the columns below and compute the correlation matrix. Save the results in the variable `cor_fit`
column= ['Calorie burned', 'Steps','Floors','Minutes Sedentary','Minutes Very Active', 'MinutesOfSleep']

In [251]:
# your answer here


#### Based on the result of the correlation matrix. What are the highly correlated features with the variable Steps?
*Hint: From the results above select the variable Steps and order your results in descending order (use Pandas sort_values function)*

In [252]:
# your answer here


# Challenge 3

#### Open the `Time_Grades.csv` file and name your dataset `time_grades`.

In [253]:
# your answer here


#### Show visually the frequency distribution of the time_grades dataset.

In [254]:
# your answer here


#### Is there a correlation between study time and Grade? Use both Pearson and Spearman correlation and comment your results.

In [255]:
# your answer here


#### Use matplotlib to visually plot Study time vs Grade. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [256]:
# your answer here


#### Could you explain the difference between Pearson and Spearman correlation? (see References in README.md) 

In [257]:
# your answer here
